# Preprocessor Notebook : Logements Sociaux, fichier RPLS annuel

Ce notebook traite le fichier Excel du RPLS annuel : données sur les logements sociaux.
Le but est de récupérer les datasets suivants, à partir du fichier XSLX téléchargé depuis le site du ministère du Développement Durable :
 - Données par régions
 - Données par départements
 - Données par EPCI
 - Données par communes

 ### Paramètres
 Ce Notebook prend des paramètres en entrée, définis sur la toute première cellule (ci-dessus).
 La cellule a le tag "parameters" ce qui permet de lui passer des valeurs via papermill.
 - filepath : le chemin vers le fichier Excel à traiter
 - model_name : le nom du modèle source

 ### Principe
 Ce notebook extrait 4 feuilles du fichier Excel d'entrée : region, departement, epci, communes. 
 Chaque feuille est chargée dans un dataFrame puis sauvegardée en .xlsx, et chargée en base de données Bronze.
 Peu de retraitement sur ces dataFrames, seul le tableau "departement" a besoin de renommer une colonne.

## Initialisation

Les cellules suivantes servent à importer les modules nécessaires et à préparer les variables communes utilisées dans les traitements.

In [ ]:
# Baseline imports
import pandas as pd
import os
import sys
import datetime

# Dirty trick to be able to import common odis modules, if the notebook is not executed from 13_odis
current_dir = os.getcwd()
parent_dir = os.path.dirname(os.getcwd())
while not current_dir.endswith("13_odis"):
    print("changing to parent dir")
    os.chdir(parent_dir)
    current_dir = parent_dir
    parent_dir = os.path.dirname(current_dir)

print(os.getcwd())
sys.path.append(current_dir)

In [ ]:
# additional imports
from common.config import load_config
from common.data_source_model import DataSourceModel
from common.utils.file_handler import FileHandler
from common.utils.interfaces.data_handler import OperationType

## Paramètres du Notebook
Paramètres pouvant être passés en input par papermill.

Seuls des types built-in semblent marcher (str, int etc), les classes spécifiques ou les objets mutables (datetime...) semblent faire planter papermill.

Doc officielle de papermill : parametrize [https://papermill.readthedocs.io/en/latest/usage-parameterize.html]

In [ ]:
# Define parameters for papermill. 
filepath = 'data/imports/logement_social/logement_social.logements_sociaux_1.xlsx'
model_name = "logement_social.logements_sociaux"


In [ ]:
# Initialize common variables
dataframes = {}
artifacts = []

start_time = datetime.datetime.now(tz=datetime.timezone.utc)
config = load_config("datasources.yaml", response_model=DataSourceModel)
model = config.get_model( model_name = model_name )
# Instantiate File Handler for file loads and dumps
handler = FileHandler()

## Traitement des données
A partir de là, on charge le fichier Excel dans Pandas et on traite les feuilles à récupérer, une par une

In [ ]:
# Load workbook to pandas
wb = pd.ExcelFile(
    filepath,
    engine = 'openpyxl'
)

In [ ]:
# Load excel sheet for Regions
sheet_name = "REGION"
keep_columns_region = [
    'LIBREG',
    'densite',
    'nb_ls',
    'tx_vac',
    'tx_mob'
]


df_region = pd.read_excel(wb, 
                    sheet_name = "REGION",
                    index_col = "REG",
                    header = 5
                    )

# df_region = df_region[keep_columns_region]
dataframes["REGION"] = df_region

region_artifact = handler.artifact_dump( df_region, "REGION", model)
artifacts.append(region_artifact)

df_region.head()

In [ ]:
# Load excel sheet for Departments
keep_columns_departments = [
    'Unnamed: 1',
    'densite',
    'nb_ls',
    'tx_vac',
    'tx_mob'
]

df_department = pd.read_excel(wb, 
                    sheet_name = "DEPARTEMENT",
                    index_col = "DEP",
                    header = 5
                    )

# df_department = df_department[keep_columns_departments]

# TODO : rename column for Unnamed: 1

dataframes["DEPARTEMENT"] = df_department

department_artifact = handler.artifact_dump( df_department, "DEPARTEMENT", model)
artifacts.append(department_artifact)

df_department.head()

In [ ]:
# Load excel sheet for EPCI
keep_columns_epci = [
    'LIBEPCI',
    'densite',
    'nb_ls',
    'tx_vac',
    'tx_mob'
]

df_epci = pd.read_excel(wb, 
                    sheet_name = "EPCI",
                    index_col = "EPCI_DEP",
                    header = 5
                    )

# df_epci = df_epci[keep_columns_epci]

dataframes["EPCI"] = df_epci

epci_artifact = handler.artifact_dump( df_epci, "EPCI", model)
artifacts.append(epci_artifact)

df_epci.head()

In [ ]:
# Load excel sheet for COMMUNES
keep_columns_communes = [
    'LIBCOM_DEP',
    'densite',
    'nb_ls',
    'tx_vac',
    'tx_mob'
]

df_communes = pd.read_excel(wb, 
                    sheet_name = "COMMUNES",
                    index_col = "DEPCOM_ARM",
                    header = 5
                    )

# df_communes = df_communes[keep_columns_communes]

dataframes["COMMUNES"] = df_communes

communes_artifact = handler.artifact_dump( df_communes, "COMMUNES", model )
artifacts.append(communes_artifact)

df_communes.head()

## Sauvegarde des métadonnées
On sauvegarde les métadonnées du processus localement, pour garder l'historique et pouvoir reprendre après erreur si besoin

In [ ]:
for artifact in artifacts:
    print(artifact.model_dump( mode = "json" ))

preprocess_metadata = handler.dump_metadata(
    model = model,
    operation = OperationType.PREPROCESS,
    start_time = start_time,
    complete = True,
    errors = 0,
    artifacts = artifacts,
    pages = []
)

## Chargement en couche Bronze
On charge un engine SQLAchemy pour charger tous les datasets en base

In [ ]:
from dotenv import dotenv_values
import sqlalchemy
from sqlalchemy import text

# prepare db client
vals = dotenv_values()

conn_str = "postgresql://{}:{}@{}:{}/{}".format(
    vals["PG_DB_USER"],
    vals["PG_DB_PWD"],
    vals["PG_DB_HOST"],
    vals["PG_DB_PORT"],
    vals["PG_DB_NAME"]
)

dbengine = sqlalchemy.create_engine(conn_str)

In [ ]:
# insert all to bronze
# make the final table name lowercase to avoid issues in Postgre

for name, dataframe in dataframes.items():

    subtable_name = f"{model.table_name}_{name.lower()}"
    query_str = f"DROP TABLE IF EXISTS bronze.{subtable_name} CASCADE"

    # dropping existing table with cascade
    with dbengine.connect() as con:
        print(f"Dropping if exists: {subtable_name}")
        result = con.execute(text(query_str))
        con.commit()

    print(f"Inserting DataFrame {subtable_name}")
    dataframe.to_sql(
        name = subtable_name,
        con = dbengine,
        schema = 'bronze',
        index = True,
        if_exists = 'replace'
    )
